In [ ]:
import os
import glob
import shutil
import pandas as pd

import pymdwizard
from pymdwizard.core import xml_utils
from pymdwizard.core.xml_utils import xml_node
from pymdwizard.core import spatial_utils

In [ ]:
## Metadata record to be edited
md_fname = <<path to metadata file>>
## A crosswalk table to get from the attribute label to the full scientific name that will be appended 
## to the Attribute Description element
fname = <<path to crosswalk table>>

In [ ]:
## Read in the crosswalk table to a pandas dataframe
lookup_df = pd.read_excel(fname, "Sheet1")
lookup_df

In [ ]:
## A lookup function that extracts the scientific name from the crosswalk table
code = 'ACERCIR'
def lookup(code):
    return lookup_df.loc[lookup_df['Code'] == code, 'Scientific name'].iloc[0]
lookup(code)


In [ ]:
## The root node of our metadata record 
md = xml_utils.fname_to_node(md_fname)
## A list of the 'attr' nodes in our metadata record
attr_list = md.xpath('eainfo/detailed/attr')

In [ ]:
## The text string that is constant throughout the applicable attribute definitions
attrdef_text = "Percent living cover in plot of"

In [ ]:
rdom = attr.xpath('attrdomv/rdom')
rdom

In [ ]:
## Loop through all applicable attributes (starting in position 3 in this case), replacing the values in Attribute Definition
## with the text from 'attrdef_text' concatenated with the scientific names looked up from the crosswalk table.
## Add the Attribute Units of Measure element for each attribute and populate with 'Percent'.
for attr in attr_list[3:]:
    attrlabl_list = attr.xpath('attrlabl')
    attrlabl_node = attr.xpath('attrlabl')[0]
    attrlabl = attrlabl_node.text
    print (attrlabl)
    sciname = lookup(attrlabl)
    
    attrdef = attr.xpath('attrdef')[0]
    new_attrdef = attrdef_text  + " " + sciname
    attrdef.text = new_attrdef
    
    rdom = attr.xpath('attrdomv/rdom')[0]
    attrunit = xml_node('attrunit', 'Percent', rdom)

In [ ]:
## Save out an XML document
xml_utils.save_to_file(md, <<path to saved metadata file>>)